In [1]:
from langchain_core.messages import HumanMessage,AIMessage,SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
model=ChatGoogleGenerativeAI(model="gemini-2.5-flash",)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


In [2]:
from langchain_core.messages import HumanMessage,AIMessage,SystemMessage
messages=[SystemMessage(content="You are precise technical mentor"),HumanMessage(content="Explain message vs prompt ")]

In [5]:
from langchain_core.prompts  import ChatPromptTemplate

prompt= ChatPromptTemplate.from_messages([("system","You are helpful assistant"),("human","{question}")])

messages=prompt.invoke({"question":"What is the best thing you think of any llm can do "})
model.invoke(messages)

AIMessage(content='While LLMs can do a dizzying array of impressive things – write code, summarize books, translate languages, brainstorm ideas, and even generate poetry – I believe the **single best thing any LLM can do is to act as a highly flexible and powerful universal language interface.**\n\nLet me break down why:\n\n1.  **Bridging the Human-Information Gap:** At its core, an LLM takes human language (your prompt) and transforms it into actionable output, also in human language. It\'s the ultimate translator between human intent and a vast, complex web of information, logic, and potential. You don\'t need to learn a specific command language, a programming syntax, or complex database queries. You just speak (or type) naturally.\n\n2.  **Unlocks Versatility:** This "language interface" capability is the foundational element that allows it to do *everything else*.\n    *   **Summarization:** It interfaces with a long document and outputs a concise version.\n    *   **Code Generati

In [7]:
import os
from typing import TypedDict, List
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, START, END

# --- 1. Define the State ---
# The State is the "memory" of our graph. 
# Here, we track a list of messages.
class State(TypedDict):
    messages: List[str]

# --- 2. Initialize the Model ---
# We use Google's Gemini Flash for speed and efficiency.
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    
)

# --- 3. Define the Nodes ---
# A Node is just a function that takes 'State' and returns an update to 'State'.
def chatbot_node(state: State):
    # Get the latest message history
    user_input = state["messages"][-1]
    
    # Call Gemini
    response = llm.invoke(user_input)
    
    # Return an update. 
    # In LangGraph, returning a dict merges it with the existing state.
    # Note: For simple TypedDict, we usually append manually, 
    # but for this "Hello World", we will just overwrite/append conceptually.
    return {"messages": state["messages"] + [response.content]}

# --- 4. Build the Graph ---
workflow = StateGraph(State)

# Add our node
workflow.add_node("chatbot", chatbot_node)

# Add Edges (The flow)
# START -> chatbot -> END
workflow.add_edge(START, "chatbot")
workflow.add_edge("chatbot", END)

# Compile into a runnable
app = workflow.compile()

# --- 5. Execution ---
initial_state = {"messages": ["Hello, explain LangGraph in one sentence."]}
result = app.invoke(initial_state)

print(f"Final Output: {result['messages'][-1]}")

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


Final Output: LangGraph is a framework built on LangChain for building robust, stateful AI agents by orchestrating their complex, multi-turn decision flows and interactions as a directed graph.
